<a href="https://colab.research.google.com/github/LUCY-JungHwaKim/CodeProject/blob/master/lotte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torch.optim as optim
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
base_path = "데이터path"


In [ ]:
# 분류할때만인듯
image = []
labels = []
import os
for file in os.listdir(base_path): ## 디렉터리 안을 리스트 해주는
  if file == 'Cleoptrea':
    for c in os.listdir(os.path.join(base_path, file)):
      if c!= 'annotations':
        image.append(c)
        labels.append('Cleoptrea')

      다 하고 나서
data = {'Images': image, 'labels':label} ## 데이터프레임으로 만들어주기
data = pd.DataFrame(data)
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data['encoded_labels'] = lb.fit_transformer(data['labels'])
data.head()

In [ ]:
batch_size = 128
validation_split = .3
shuffle_dataset = True
random_seed = 42

In [ ]:
 # split dtaset
dataset_size = len(data)
indices = list(range(dataset_size)) ## 나누기위한 리스트
split = int(np.floor(validation_split * dataset_size)) # 사이즈 정하기
if shuffle_dataset:
  np.random.seed(random_seed) ## 시드 고정
  np.rnadom.shuffle(indices) ## 섞기
train_indices, val_indices = indices[split:], indices[:split]

In [ ]:
# 근데 이거로도됨
from sklearn.model_selection import train_test_split
tr, val = train_test_slit(data.label, stratify = data.label, test_size = 0.1)

In [ ]:
## 데이터 만들기
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import Dataset
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


In [ ]:
## Transforms
# Normalization
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
## mean, std for channels


In [ ]:
## Create custom dataset
## init, len, getitem
from PIL import Image
class Arthopod_dataset(Dataset):
  def __init__(self, img_data, img_path, transform = None):
    self.img_path = img_path
    self.img_data = img_data
    self.transform = transform
  def __len__(self):
    return len(self.img_data)

  def __getitem__(self, index):
    img_name = os.path.join(self.img_path, self.img_data.loc[index, 'labels'], self.img_data.loc[index, 'Images'])
    imag = Image.open(img_name)
    image = image.convert('RGB')
    image = image.resize((300,300))

    label = torch.tensor(self.image_data.loc[index, 'encoded_labels'])
    if self.transform is not None:
      image = self.transform(image)
    return image, label

In [ ]:
dataset = Arthopod_Dataset(data, base_path, transform)
train_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, sampler = train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size, sampler = validation_sampler)

In [ ]:
## Network 쌓기
import torch.nn.Functional as F

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    # 3 input channerl, 16 output channel, 3*3 square kernel
    self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 2, padding = 1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size = 3, stride = 2, padding = 1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size = 3, stride = 2, padding = 1)
    self.conv4 = nn.Conv2d(64, 128, kernel_size = 3, stride = 2, padding = 1)
    self.pool = nn.MaxPool2d(2,2)
    self.dropout = nn.Dropout2d(0.4)
    self.batchnorm1 = nn.BatchNorm2d(16)
    self.batchnorm2 = nn.Batchnorm2d(32)
    self.batchnorm3 = nn.Batchnorm2d(64)
    self.fc1 = nn.Linear(64*5*5, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 4)

  def forward(self, x):
    x = self.batchnorm1(F.relu(self.conv1(x)))
    x = self.batchnorm2(F.relu(self.conv2(x)))
    x  = self.dropout(self.batchnorm2(self.pool(x))) ## 뭐든 배치정규화 하기.. x하고나서
    x = self.batchnorm3(self.pool(F.relu(self.conv3(x)))) ## relu 후에 pool
    x = self.dropout(self.conv4(x))
    x = x.view(-1, 64*5*5) ## flatten
    x = self.dropout(self.fc1(x))
    x = self.dropout(self.fc2(x))
    x = F.log_softmax(self.fc3(x), dim = 1)
    return x

In [ ]:
model = Net() ## cpu
model = Net().to(device) ## gpu
print(model)

In [ ]:
## Loss
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
def accuracy(out, labeels):
  _,pred = torch.max(out, dim = 1)
  return torch.sum(pred == labels).item()

In [ ]:
## Train
n_epochs = 12
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_loader)

for epoch in range(1, n_epochs+1):
  running_loss = 0.0
  scheduler.steo(epoch)
  correct = 0
  total = 0

  print(epoch)
  ## 학습
  for batch_idx, (data_, target_) in enumerate(train_loader):
    data_, target_ = data_.to(device), target_.to(device)

    optimizer.zero_grad() ## 파라미터 초기화
    outputs = model(data_)

    loss = criterion(outputs, target_)
    loss.backward()
    optimizer.step()

    running_loss += looss.item()
    _, pred = torch.max(outputs, dim = 1)
    correct += torh.sum(pred == target_).item() ## 맞은것만
    total += target_.size(0)  # r개수

    ## 배치 사이즈별로 출력
    if (batch_idx)  % 20 == 0:
      print(머시깽)

  train_acc.append(100  * (correct/total))
  train_loss.append(running_loss/total_step)

  ## valid
  batch_loss = 0
  total_t = 0
  correct_t = 0
  with torch.no_grad():
    model.eval() ## 평가모드
    for data_t, target_t in (validation_loader):
      data_t, target_t = data_t.to(device), target_t.to(device)

      output_t = model(data_t)
      loss_t = criterion(outputs_t, target_t)
      batch_loss += loss_t.item()
      _, pred_t = torch.max(outputs_t, dim = 1)
      correct_t = torch.sum(pred_t == target_t).item()
      total_t += target_t.size(0)

    val_acc.append(100 * (correct_T/total_t))
    val_loss.append(batch_loss/len(validation_loader))

    network_learned = batch_loss < valid_loss_min

    if network_learned : ##감소했다면
      valid_loss_min = batch_loss
      torch.save(model.state_dict(), 'model_classification_tutorial.pt')

  model.train() ## 학습 모드, 다음 에포크에서도 학습 해야하니깐


In [ ]:
plt.plot(train_loss)
plt.plot(val_loss), acc도 가능

In [ ]:
## 학습된 네트워크 가지고오기
model.load_state_dict(torch.load('model_classification_tutorial.pt'))


In [ ]:
## evaluation
dataiter = iter(validation_loader)
images, labels = dataiter.next()
image.unsqueeze_(0) ## 차원삽
# https://stackoverflow.com/questions/57237352/what-does-unsqueeze-do-in-pytorch

#  Transfer learning

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os


In [6]:
transforms.info()

AttributeError: module 'torchvision.transforms' has no attribute 'info'

In [7]:
## 데이터 불러오기
data_transforms = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
    ]),
    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
    ])

}

data_dir = 'path'
image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x : torch.utils.data.DataLoader(image_datasets[x], batch_size = 4, shuffle=True, num_workers = 4) for x in ['train', 'val']}

dataset_sizes = {x : len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

FileNotFoundError: [Errno 2] No such file or directory: 'path/train'

In [ ]:
## 모델 학습
import copy
def train_model(model, criterion, optimizer, scheduler, num_epochs = 25):

  best_model_wts = copy.deepcopy(model.state_dict())

  best_acc = 0.0

  for epoch in range(num_epochs):
    for phase in ['train', 'val']:
      if phase == 'train':
        model.train()
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0.0

      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'): ## 학습시에만 연산 기록 추적
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels)

          if phase == 'train':
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      if phase == 'train':
        scheduler.step()

      epoch_loss =  running_loss / dataset_sizes[phase]
      epoch_acc = running_correccts.doupble() / dataset_sizes[phase]